In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np

import pylinac

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys import _losslessjpeg as lljpeg
from pymedphys._wlutz import iview as _iview
from pymedphys._wlutz import imginterp as _imginterp
from pymedphys._wlutz import findfield as _findfield
from pymedphys._wlutz import findbb as _findbb
from pymedphys._wlutz import reporting as _reporting
from pymedphys._wlutz import utilities as _utilities

In [ ]:
def read_image(path):
    return lljpeg.imread(path)

In [ ]:
path = pathlib.Path(
    r'\\tunnel-nbcc-pdc\Physics\NBCC-DataExchange\iViewDB\patient_MVISO\MV_IMAGES\img_1.3.46.423632.301843.1579156836.86\00022.1.3.46.423632.301843.1579156836.86.jpg'
)

In [ ]:
edge_lengths = [20, 24]
penumbra = 2
bb_diameter = 8

In [ ]:
pylinac_interpolated_pixel_size = 0.05

In [ ]:
raw_img = read_image(path)
x, y, img = _iview.iview_image_transform(raw_img)

field = _imginterp.create_interpolated_field(x, y, img)
initial_centre = _findfield.get_centre_of_mass(x, y, img)

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(x, y, img, shading='nearest')

ax.axis('equal')
ax.set_xlim([-30,30])
ax.set_ylim([-30,30])

In [ ]:
field_centre, field_rotation = _findfield.field_centre_and_rotation_refining(
    field, edge_lengths, penumbra, initial_centre
)

In [ ]:
bb_centre = _findbb.optimise_bb_centre(
    field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation
)

In [ ]:
fig = _reporting.image_analysis_figure(
    x,
    y,
    img,
    bb_centre,
    field_centre,
    field_rotation,
    bb_diameter,
    edge_lengths,
    penumbra,
)

In [ ]:
pylinac.winston_lutz.WLImage

In [ ]:
pylinac.image.ArrayImage

In [ ]:
class CustomWLImage(pylinac.image.ArrayImage):
    def __init__(self, array, *, dpi=None, sid=None, dtype=None):
        super().__init__(array, dpi=dpi, sid=sid, dtype=dtype)
        self.check_inversion_by_histogram(percentiles=(0.01, 50, 99.99))
        self._clean_edges()
        self.ground()
        self.normalize()
        self._field_cax = None
        self.rad_field_bounding_box = None
        self._bb = None
        
    def _run_field_finding(self):
        self._field_cax, self.rad_field_bounding_box = self._find_field_centroid()
        
    
    @property
    def field_cax(self):
        if self._field_cax is None:
            self._run_field_finding()
            
        return self._field_cax
    
    @property
    def bb(self):
        if self._bb is None:
            if self.rad_field_bounding_box is None:
                self._run_field_finding()
            
            self._bb = self._find_bb()
        
        return self._bb
                
        
    __repr__ = pylinac.winston_lutz.WLImage.__repr__
    _clean_edges = pylinac.winston_lutz.WLImage._clean_edges
    _find_field_centroid = pylinac.winston_lutz.WLImage._find_field_centroid
    _find_bb = pylinac.winston_lutz.WLImage._find_bb

In [ ]:
centralised_straight_field = _utilities.create_centralised_field(
    field, field_centre, field_rotation
)

half_x_range = edge_lengths[0] / 2 + penumbra * 3
half_y_range = edge_lengths[1] / 2 + penumbra * 3

pylinac_x = np.arange(-half_x_range, half_x_range + pylinac_interpolated_pixel_size, pylinac_interpolated_pixel_size)
pylinac_y = np.arange(-half_y_range, half_y_range + pylinac_interpolated_pixel_size, pylinac_interpolated_pixel_size)

pylinac_xx, pylinac_yy = np.meshgrid(pylinac_x, pylinac_y)
centralised_image = centralised_straight_field(pylinac_xx, pylinac_yy)

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(pylinac_x, pylinac_y, centralised_image, shading='nearest')

ax.axis('equal')

In [ ]:
wl_image = CustomWLImage(centralised_image)

In [ ]:
centralised_pylinac_field_centre = [
    wl_image.field_cax.x * pylinac_interpolated_pixel_size - half_x_range,
    wl_image.field_cax.y * pylinac_interpolated_pixel_size - half_y_range,
]

In [ ]:
pylinac_field_centre = _utilities.transform_point(
    centralised_pylinac_field_centre, field_centre, field_rotation
)

In [ ]:
centralised_pylinac_bb_centre = [
    wl_image.bb.x * pylinac_interpolated_pixel_size - half_x_range,
    wl_image.bb.y * pylinac_interpolated_pixel_size - half_y_range,
]

pylinac_bb_centre = _utilities.transform_point(
    centralised_pylinac_bb_centre, field_centre, field_rotation
)

In [ ]:
fig = _reporting.image_analysis_figure(
    x,
    y,
    img,
    pylinac_bb_centre,
    pylinac_field_centre,
    field_rotation,
    bb_diameter,
    edge_lengths,
    penumbra,
)

In [ ]:
pylinac.__version__